# Export PBI Metatdata to Lakehouse

Version==1.0.1

In [ ]:
%pip install semantic-link-labs==0.10.1


### Configuration:


In [ ]:
#filter by workspace list
workspace_list=[]

#include personal workspace
include_personal_workspace=False

#filter by PP3 workspace
only_PP3=True

In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!

import sempy
import sempy.fabric as fabric
import pandas as pd
import json
import re
import sempy_labs as labs
from sempy_labs import admin as labs_admin
from sempy_labs import report as labs_report



def sanitize_name(name: str) -> str:
    """
    Replace any character in the input string that is not a letter, number, or underscore with '_'.
    """
    return re.sub(r'[^a-zA-Z0-9_]', '_', name)
def clean_name(dataframe):
    dataframe.columns = [col.replace(" ", "_") for col in dataframe.columns]

lakehouse="LH_"+sanitize_name(notebookutils.runtime.context['currentNotebookName'])


#check if lakehouse exist
df_items =fabric.list_items()
lakehouse_exists = df_items['Display Name'].eq(lakehouse).any()
if lakehouse_exists:
    print("Lakehouse already exists.")
    lhid = ( df_items[df_items['Display Name'].eq(lakehouse)&df_items['Type'].eq('Lakehouse')].iloc[0, 0])
else:
    print("Lakehouse does not exist. Creating...")
    lhid = fabric.create_lakehouse(lakehouse)


all_capacities=labs_admin.list_capacities()
clean_name(all_capacities)

labs.save_as_delta_table(dataframe=all_capacities, delta_table_name="Capacities", write_mode="overwrite",merge_schema=True,lakehouse=lakehouse)

print(all_capacities.columns)




In [ ]:
 
    
all_workspaces_df = labs_admin.list_workspaces()
clean_name(all_workspaces_df)
all_workspaces_df['Comment']=''
all_capacities=all_capacities[['Capacity_Id','Capacity_Name','Sku','Region']]
print(all_capacities.columns)
print(all_workspaces_df.columns)

all_workspaces_df=pd.merge(all_workspaces_df,all_capacities,left_on='Capacity_Id',right_on='Capacity_Id',how='left')

def filter_workspace(ws_df, type):
    return ws_df.drop(ws_df[(ws_df.Type == type)].index)


all_workspaces_df=filter_workspace(all_workspaces_df,'AdminWorkspace')
if workspace_list:
    all_workspaces_df=all_workspaces_df[all_workspaces_df["Name"].isin(workspace_list)]


#filter out PP3
if only_PP3:
    all_workspaces_df = all_workspaces_df[all_workspaces_df['Sku'] == 'PP3'].copy()

if not include_personal_workspace:
    all_workspaces_df=filter_workspace(all_workspaces_df,'Personal')
    

#all_workspaces_df=all_workspaces_df.drop(columns=["Capacity Id"])
labs.save_as_delta_table(dataframe=all_workspaces_df, delta_table_name="Workspaces", write_mode="overwrite",merge_schema=True,lakehouse=lakehouse)


    


In [ ]:
# items_list=[]
# for _, ws_row in all_workspaces_df.iterrows():
#     ws_name=ws_row['Name']
#     #get items
#     df_items=labs_admin.list_items(workspace=ws_name)
#     df_items["Workspace_Name"]=ws_name
#     items_list.append(df_items)

# df_all_items=pd.concat(items_list,ignore_index=True)
# labs.save_as_delta_table(dataframe=df_all_items,delta_table_name="WorkspaceItems",write_mode="overwrite",merge_schema=True,lakehouse=lakehouse)

In [ ]:
df_items = labs_admin.list_items()
# clean_name(df_items)
# items_list=[]
# for _, ws_row in all_workspaces_df.iterrows():
#     ws_name=ws_row['Name']
#     #get items
#     df_items=labs_admin.list_items(workspace=ws_name)
#     df_items["Workspace_Name"]=ws_name
#     items_list.append(df_items)

# df_all_items=pd.concat(items_list,ignore_index=True)
labs.save_as_delta_table(dataframe=df_items,delta_table_name="WorkspaceItemsAll",write_mode="overwrite",merge_schema=True,lakehouse=lakehouse)

In [ ]:
df_semantic_models = labs_admin.list_datasets()
df_semantic_models["Upstream Datasets"] = df_semantic_models["Upstream Datasets"].astype(str)
df_semantic_models["Users"] = df_semantic_models["Users"].astype(str)
# df_large_models = df_semantic_models[df_semantic_models['Target Storage Mode'] == 'PremiumFiles'].copy()
# #df_large_models = df_large_models[df_large_models["Workspace Id"].isin(all_workspaces_df['Id'])]
# import uuid
# import math
# for _, large_model_row in df_large_models.iterrows():
#     large_model_id = large_model_row["Dataset Id"]
#     large_model_workspace_id = uuid.UUID(large_model_row["Workspace Id"])

#     if not labs.is_default_semantic_model(dataset=large_model_id, workspace=large_model_workspace_id):
#         model_size = labs.get_semantic_model_size(dataset=large_model_id, workspace=large_model_workspace_id)
#         df_large_models["Model Size"] = round(model_size/(1024*1024*1024),3)
    
#     # You can add any processing logic here for each large model
#     print(f"Processing large model: {large_model_id} in workspace: {large_model_workspace_id}")



labs.save_as_delta_table(dataframe=df_semantic_models,delta_table_name="SemanticModels",write_mode="overwrite",merge_schema=True,lakehouse=lakehouse)

In [ ]:
def get_model_prop(workspace):
    print(f"process workspace {workspace}")
    server = fabric.create_tom_server(readonly=True, workspace=workspace)
    print(f"connect to {workspace}")
    results=[]
    for d in server.Databases:
        r={}
        print(f"{d.Name} {d.ID}  {round(d.EstimatedSize/(1024*1024*1024),3)}")
        flist=""
        for a in dir(d):
            try:
                if a.startswith("get_") and a != "get_SurrogatePairBehavior" :
                    value=eval(f"d.{a}()")
                    name=a.replace("get_","")
                    if type(value) not in [str,int]:
                        value=str(value)
                    # print(f"{name} {value} {type(value)}")
                    flist+=name+","
                    r[name]=value
            except Exception as e:
                print(e)
        #print(flist)
        r['Workspace']=workspace
        results.append(r)
    return pd.DataFrame(results)
all_models=[]
 
for index, ws_row in all_workspaces_df.iterrows():
    ws_name=ws_row['Name']
    ws_id=ws_row['Id']
    try:
        models_meta=get_model_prop(ws_name)
        all_models.append(models_meta)
    except Exception as e:
        all_workspaces_df.at[index,'Comment']=str(e)
        #print(e)
        print(f"failed to process workspace {ws_name} {ws_id}")
# print(all_models)
df_all_items=pd.concat(all_models,ignore_index=True)
# print(df_all_items)
labs.save_as_delta_table(dataframe=all_workspaces_df, delta_table_name="Workspaces", write_mode="overwrite",merge_schema=True,lakehouse=lakehouse)
if df_all_items is not None:
    labs.save_as_delta_table(dataframe=df_all_items,delta_table_name="SemanticModelsXmla",write_mode="overwrite",merge_schema=True,lakehouse=lakehouse)

In [ ]:
sql=f'''

select m.Workspace_Id,x.Workspace ,m.Dataset_Id, m.Dataset_Name, m.Target_Storage_Mode,x.EstimatedSize/(1024*1024*1024) as Size from 
{lakehouse}.SemanticModelsXmla x , {lakehouse}.SemanticModels m where x.ID=m.Dataset_Id
 and  m.Target_Storage_Mode = 'PremiumFiles'
'''
print(sql)
df = spark.sql(sql)
display(df)

workspace license info pro:

process workspace test model migration new
The '<euii>freeuser1@lgjtest.onmicrosoft.com</euii>' user does not have permission to call the Discover method.


capacity is not available
Server '977D1839-3AB0-4005-B31B-2B6CABBE0745' is not found.

